## Sommaire
### Mission 1: Préparation des données
##### I.Nettoyage des données
<br/>I.1.[Transactions](#transactions)
<br/>I.2.[Customers](#customers)
<br/>I.3.[Products](#products)
##### II.Finalisation du fichier consolidé
<br/>II.1.[Jointure des dataframes](#join)
<br/>II.2.[Variables supplémentaires](#new)

In [194]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import datetime as dt
import seaborn as sns
from matplotlib import pylab
import matplotlib.dates as mdates
import datetime as dt
import scipy.stats as st
import statsmodels.api as sm
import matplotlib.axes as ax
from matplotlib.pylab import arange, plot

In [195]:
#importation des fichiers
transactions = pd.read_csv('transactions.csv')
products =  pd.read_csv('products.csv')
customers = pd.read_csv('customers.csv')

## Nettoyage du dataframe transactions <a id='transactions'></a>

<br/> Nous allons nettoyer le fichier transactions regroupant les informations sur les sessions de ventes

In [196]:
transactions.shape

(337016, 4)

In [197]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337016 entries, 0 to 337015
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     337016 non-null  object
 1   date        337016 non-null  object
 2   session_id  337016 non-null  object
 3   client_id   337016 non-null  object
dtypes: object(4)
memory usage: 10.3+ MB


<br/> Pas de valeur nulle dans le fichier.

In [198]:
transactions.describe(include='all')

,id_prod,date,session_id,client_id
count,337016,337016,337016,337016
unique,3266,336855,169195,8602
top,1_369,test_2021-03-01 02:30:02.237413,s_0,c_1609
freq,1081,13,200,12855


<br/> Il y a une session s_0 qui correspond sûrement à une session test, ainsi qu'une date débutant par "test". Si cela est avéré, nous allons devoir les supprimer.

### Suppression des valeurs dupliquées

In [199]:
transactions.duplicated().sum()

126

In [200]:
duplicated_transa = transactions.duplicated()
transactions[duplicated_transa]

,id_prod,date,session_id,client_id
34387,T_0,test_2021-03-01 02:30:02.237443,s_0,ct_0
54813,T_0,test_2021-03-01 02:30:02.237412,s_0,ct_1
57261,T_0,test_2021-03-01 02:30:02.237439,s_0,ct_1
58802,T_0,test_2021-03-01 02:30:02.237429,s_0,ct_0
60170,T_0,test_2021-03-01 02:30:02.237446,s_0,ct_0
...,...,...,...,...
332594,T_0,test_2021-03-01 02:30:02.237445,s_0,ct_0
332705,T_0,test_2021-03-01 02:30:02.237423,s_0,ct_1
332730,T_0,test_2021-03-01 02:30:02.237421,s_0,ct_1
333442,T_0,test_2021-03-01 02:30:02.237431,s_0,ct_1


<br/> Le dataset contient 126 valeurs dupliquées qui correspondent à des sessions de test comme vu précedemment. Nous pouvons donc les supprimer.

In [201]:
indexNames = transactions[transactions['id_prod'] == 'T_0' ].index
transactions.drop(indexNames , inplace=True)
transactions.sort_values(by = 'id_prod', ascending=False)

,id_prod,date,session_id,client_id
65672,2_99,2021-11-21 01:53:46.967570,s_122697,c_5828
139355,2_99,2021-07-11 20:56:49.820935,s_61009,c_7135
326898,2_98,2021-03-08 21:10:32.250919,s_3637,c_5967
209271,2_97,2021-08-15 20:16:10.343556,s_76200,c_3331
284189,2_97,2021-04-06 19:09:32.897796,s_16930,c_5899
...,...,...,...,...
23273,0_0,2021-05-11 22:29:41.693404,s_33228,c_1893
23349,0_0,2022-01-16 09:24:40.731530,s_150579,c_8502
116470,0_0,2022-01-20 14:36:47.654060,s_152617,c_1609
23375,0_0,2021-10-24 10:29:51.488421,s_109401,c_4973


In [202]:
transactions.duplicated().sum()

0

### Conversion du type de la colonne "date"

<br/> Dans le cadre de l'analyse, il est nécessaire de transformer la colonne "date" en datetime afin de la manipuler plus facilement.

In [203]:
transactions.dtypes

id_prod       object
date          object
session_id    object
client_id     object
dtype: object

In [204]:
transactions['date'] = pd.to_datetime(transactions['date'])
transactions.dtypes

id_prod               object
date          datetime64[ns]
session_id            object
client_id             object
dtype: object

<br/> La colonne "date" est un timestamp regroupant la date et l'heure. Séparons ces deux informations dans des colonnes distinctes.

In [205]:
transactions.rename(columns={'date': 'date_time'}, inplace=True)
transactions['hour']=transactions['date_time'].dt.hour
transactions['date'] = transactions['date_time'].dt.date
transactions['time'] = transactions['date_time'].dt.time
transactions=transactions.drop(columns='date_time')
transactions

,id_prod,session_id,client_id,hour,date,time
0,0_1483,s_18746,c_4450,18,2021-04-10,18:37:28.723910
1,2_226,s_159142,c_277,1,2022-02-03,01:55:53.276402
2,1_374,s_94290,c_4270,15,2021-09-23,15:13:46.938559
3,0_2186,s_105936,c_4597,3,2021-10-17,03:27:18.783634
4,0_1351,s_63642,c_1242,20,2021-07-17,20:34:25.800563
...,...,...,...,...,...,...
337011,1_671,s_40720,c_3454,12,2021-05-28,12:35:46.214839
337012,0_759,s_50568,c_6268,0,2021-06-19,00:19:23.917703
337013,0_1256,s_7219,c_4137,17,2021-03-16,17:31:59.442007
337014,2_227,s_112349,c_5,16,2021-10-30,16:50:15.997750


<br/> Le fichier "transactions" est désormais nettoyé !

## Nettoyage du dataframe customers <a id='customers'></a>

<br/> Le fichier "customers" contient le genre et la date de naissance des clients selon leur id.

In [206]:
customers.shape

(8623, 3)

In [207]:
customers.head()

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943


In [208]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8623 non-null   object
 1   sex        8623 non-null   object
 2   birth      8623 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 202.2+ KB


<br/> Pas de valeurs nulle dans le fichier.

### Suppression des clients test

<br/> Lors du nettoyage du fichier "transactions", nous avons remarqué des sessions de test associé aux clients test "ct_0" et "ct_1". Supprimons les du fichier.

In [209]:
#Création d'un index pour récupérer la ligne correspondant à l'id client "ct_0"
id_test0 = customers[customers['client_id'] == 'ct_0' ].index
customers.drop(id_test0 , inplace=True)
customers.sort_values(by = 'client_id', ascending=False)

,client_id,sex,birth
8494,ct_1,m,2001
7358,c_999,m,1964
2145,c_998,m,2001
94,c_997,f,1994
2788,c_996,f,1970
...,...,...,...
3426,c_1001,m,1982
8472,c_1000,f,1966
2137,c_100,m,1992
6894,c_10,m,1956


In [210]:
#Création d'un index pour récupérer la ligne correspondant à l'id client "ct_1"
id_test1 = customers[customers['client_id'] == 'ct_1' ].index
customers.drop(id_test1 , inplace=True)
customers.sort_values(by = 'client_id', ascending=False)

,client_id,sex,birth
7358,c_999,m,1964
2145,c_998,m,2001
94,c_997,f,1994
2788,c_996,f,1970
7004,c_995,m,1955
...,...,...,...
3426,c_1001,m,1982
8472,c_1000,f,1966
2137,c_100,m,1992
6894,c_10,m,1956


In [211]:
customers.duplicated().sum()

0

<br/> Aucune valeur dupliquée dans le fichier.

In [212]:
customers.isnull().sum() 

client_id    0
sex          0
birth        0
dtype: int64

<br/> Pas de valeurs nulles dans le fichier

<br/> Le fichier "customers" est nettoyé !

## Nettoyage du dataframe products <a id='products'></a>

In [213]:
products.shape

(3287, 3)

In [214]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id_prod  3287 non-null   object 
 1   price    3287 non-null   float64
 2   categ    3287 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 77.2+ KB


<br/> Aucune valeur nulle dans le fichier.

In [215]:
products.describe(include='all')

,id_prod,price,categ
count,3287,3287.000000,3287.000000
unique,3287,NaN,NaN
top,2_113,NaN,NaN
freq,1,NaN,NaN
mean,NaN,21.856641,0.370246
std,NaN,29.847908,0.615387
min,NaN,-1.000000,0.000000
25%,NaN,6.990000,0.000000
50%,NaN,13.060000,0.000000
75%,NaN,22.990000,1.000000


<br/> Nous remarquons la présence d'un prix négatif (-1) dans le fichier. Voyons à quelle produit il est associé.

In [216]:
products.sort_values(by='id_prod',ascending=False)

,id_prod,price,categ
731,T_0,-1.00,0
3188,2_99,84.99,2
3088,2_98,149.74,2
2698,2_97,160.99,2
2576,2_96,47.91,2
...,...,...,...
922,0_1000,6.84,0
663,0_100,20.60,0
2691,0_10,17.95,0
803,0_1,10.99,0


<br/> Il s'agit du produit "T_0" qui est associé au prix négatif vu plus haut, nous allons donc le supprimer.

In [217]:
#Création d'un index pour récupérer la ligne correspondant à l'id produit "T_0"
indexNames = products[products['id_prod'] == 'T_0' ].index
products.drop(indexNames , inplace=True)
products.sort_values(by = 'id_prod', ascending=False)

,id_prod,price,categ
3188,2_99,84.99,2
3088,2_98,149.74,2
2698,2_97,160.99,2
2576,2_96,47.91,2
966,2_95,98.99,2
...,...,...,...
922,0_1000,6.84,0
663,0_100,20.60,0
2691,0_10,17.95,0
803,0_1,10.99,0


In [218]:
products.duplicated().sum()

0

<br/> Aucune valeur dupliquée dans le fichier.

<br/>Il est nécessaire de transformer la colonne correspondant à la catégorie de produits en type "category" afin de faciliter sa manipulation.

In [219]:
products['categ'] = products['categ'].astype('category')
products.dtypes

id_prod      object
price       float64
categ      category
dtype: object

<br/>Verifions une dernière fois qu'il n'y a aucun prix inférieur à 0 qui nous aurait échappé.

In [220]:
products.sort_values(by='price',ascending=False)

,id_prod,price,categ
947,2_2,300.00,2
724,2_76,254.44,2
394,2_158,247.22,2
1436,2_167,236.99,2
2779,2_30,233.54,2
...,...,...,...
1530,0_1620,0.80,0
1211,0_1844,0.77,0
370,0_120,0.66,0
2272,0_528,0.62,0


<br/> Le fichier "products" est désormais nettoyé ! Passons à la jointure des fichiers afin de créer un dataframe consolidé.

## Création des jointures entre les différentes dataframes <a id='join'></a>

In [221]:
transac_prod = pd.merge(products, transactions, on='id_prod', how='outer')
transac_prod

,id_prod,price,categ,session_id,client_id,hour,date,time
0,0_1421,19.99,0,s_33709,c_7954,23.0,2021-05-12,23:26:29.885083
1,0_1421,19.99,0,s_154957,c_254,11.0,2022-01-25,11:27:39.357012
2,0_1421,19.99,0,s_144068,c_5770,22.0,2022-01-02,22:29:33.796003
3,0_1421,19.99,0,s_2187,c_1455,17.0,2021-03-05,17:10:40.265676
4,0_1421,19.99,0,s_13112,c_2768,13.0,2021-03-29,13:55:04.422637
...,...,...,...,...,...,...,...,...
336833,0_2245,NaN,NaN,s_16936,c_4167,19.0,2021-04-06,19:59:19.462288
336834,0_2245,NaN,NaN,s_13738,c_7790,23.0,2021-03-30,23:29:02.347672
336835,0_2245,NaN,NaN,s_128815,c_6189,14.0,2021-12-03,14:14:40.444177
336836,0_2245,NaN,NaN,s_26624,c_1595,18.0,2021-04-27,18:58:47.703374


<br/> La jointure a été faite sur l'id du produit afin d'identifier facilement les produits présents dans la base de données "products" mais qui n'ont pas été achetés lors d'une session. Ici, nous remarquons déjà des valeurs nulles correspondant à l'id produit 0_2245.

In [222]:
transac_prod[transac_prod['id_prod'].str.contains('0_2245')][['id_prod', 'session_id', 'price','categ']]

,id_prod,session_id,price,categ
336735,0_2245,s_49705,NaN,NaN
336736,0_2245,s_49323,NaN,NaN
336737,0_2245,s_124474,NaN,NaN
336738,0_2245,s_172304,NaN,NaN
336739,0_2245,s_3,NaN,NaN
...,...,...,...,...
336833,0_2245,s_16936,NaN,NaN
336834,0_2245,s_13738,NaN,NaN
336835,0_2245,s_128815,NaN,NaN
336836,0_2245,s_26624,NaN,NaN


<br/> Nous savons que le 1er chiffre de l'id produit correspond à la catégorie à laquelle il appartient. Nous pouvons donc ici ajouter la valeur 0 dans la colonne "categ" et remplacer les prix par la moyenne de prix de cette catégorie afin d'avoir un prix cohérent.

In [223]:
# Ajout de la valeur 0 dans la colonne catégorie
transac_prod['categ'] = transac_prod[['categ']].fillna(value=0)

In [224]:
#Calcul de la moyenne de prix par catégorie de produits
products.groupby(by='categ').mean()

,price
categ,
0,11.732795
1,25.531421
2,108.354686


In [225]:
# Ajout de la moyenne de prix dans la colonne correspondante 0
transac_prod['price'] = transac_prod[['price']].fillna(value=11.73)

In [226]:
transac_prod[transac_prod['id_prod'].str.contains('0_2245')][['id_prod', 'session_id', 'price','categ']]

,id_prod,session_id,price,categ
336735,0_2245,s_49705,11.73,0
336736,0_2245,s_49323,11.73,0
336737,0_2245,s_124474,11.73,0
336738,0_2245,s_172304,11.73,0
336739,0_2245,s_3,11.73,0
...,...,...,...,...
336833,0_2245,s_16936,11.73,0
336834,0_2245,s_13738,11.73,0
336835,0_2245,s_128815,11.73,0
336836,0_2245,s_26624,11.73,0


<br/> Vérifions qu'il ne reste pas de valeur nulle.

In [227]:
transac_prod[transac_prod.isnull().any(axis=1)]

,id_prod,price,categ,session_id,client_id,hour,date,time
19685,0_1016,35.06,0,NaN,NaN,NaN,NaN,NaN
31123,0_1780,1.67,0,NaN,NaN,NaN,NaN,NaN
80898,0_1062,20.08,0,NaN,NaN,NaN,NaN,NaN
88224,0_1119,2.99,0,NaN,NaN,NaN,NaN,NaN
90039,0_1014,1.15,0,NaN,NaN,NaN,NaN,NaN
93444,1_0,31.82,1,NaN,NaN,NaN,NaN,NaN
114536,0_1318,20.92,0,NaN,NaN,NaN,NaN,NaN
125973,0_1800,22.05,0,NaN,NaN,NaN,NaN,NaN
144152,0_1645,2.99,0,NaN,NaN,NaN,NaN,NaN
159269,0_322,2.99,0,NaN,NaN,NaN,NaN,NaN


In [228]:
invendu=transac_prod[transac_prod.isnull().any(axis=1)]

<br/> Ce sont les produits qui n'ont pas été relié à une session, ils seront donc supprimé lors de la jointure avec le troisième fichier.

In [229]:
transac_prod.duplicated().sum()

0

<br/> Aucune valeur dupliquée dans le dataframe.

<br/>Le fichier est nettoyé, nous pouvons le joindre avec le troisième fichier et former le dataframe consolidé que l'on nommera "overview".

In [230]:
overview=pd.merge(transac_prod, customers)

overview.sort_values(by='date', inplace=True)
overview=overview.reset_index(drop=True)

overview

,id_prod,price,categ,session_id,client_id,hour,date,time,sex,birth
0,0_2007,9.99,0,s_201,c_6906,10.0,2021-03-01,10:06:06.726895,m,1986
1,0_2096,14.84,0,s_311,c_6823,14.0,2021-03-01,14:54:32.544060,m,1958
2,0_1627,3.99,0,s_72,c_6823,2.0,2021-03-01,02:58:10.580555,m,1958
3,2_108,63.36,2,s_314,c_5042,15.0,2021-03-01,15:03:55.782366,m,1995
4,1_446,23.92,1,s_109,c_5890,4.0,2021-03-01,04:34:54.577506,m,1957
...,...,...,...,...,...,...,...,...,...,...
336811,0_1152,10.70,0,s_172145,c_332,10.0,2022-02-28,10:31:46.010138,f,1973
336812,0_2134,5.57,0,s_171999,c_903,3.0,2022-02-28,03:26:03.374741,f,1963
336813,0_2293,4.99,0,s_171940,c_2048,1.0,2022-02-28,01:05:47.900681,m,1987
336814,0_1140,3.73,0,s_171986,c_8547,2.0,2022-02-28,02:43:12.724027,f,1977


<br/> Comme fait précedemment, nous pouvons convertir la colonne "date" en datetime.

In [231]:
overview['date'] = pd.to_datetime(overview['date'])
overview.dtypes

id_prod               object
price                float64
categ                  int64
session_id            object
client_id             object
hour                 float64
date          datetime64[ns]
time                  object
sex                   object
birth                  int64
dtype: object

## Ajout de nouvelles variables nécessaires à l'analyse <a id='new'></a>

In [232]:
# Ajout du nombre de produits total achetés par clients
overview['nb_sales_clients']=overview['client_id'].value_counts()[overview['client_id']].tolist()
overview.sort_values(by = 'nb_sales_clients')

,id_prod,price,categ,session_id,client_id,hour,date,time,sex,birth,nb_sales_clients
5484,2_185,43.99,2,s_2670,c_6286,17.0,2021-03-06,17:49:46.651811,f,1998,1
263669,1_64,19.81,1,s_136405,c_712,20.0,2021-12-18,20:54:25.725059,f,1967,1
64761,1_385,25.99,1,s_31965,c_6397,4.0,2021-05-09,04:41:09.707797,m,1992,1
95525,0_1480,4.15,0,s_47613,c_8140,11.0,2021-06-12,11:30:18.492800,m,1971,1
108993,1_318,17.24,1,s_54466,c_3923,7.0,2021-06-27,07:44:42.774335,f,2004,1
...,...,...,...,...,...,...,...,...,...,...,...
204875,0_1113,16.99,0,s_104849,c_1609,22.0,2021-10-14,22:53:25.284484,m,1980,12855
204864,0_1465,3.99,0,s_104616,c_1609,11.0,2021-10-14,11:39:19.983587,m,1980,12855
265321,0_2135,4.99,0,s_137086,c_1609,5.0,2021-12-20,05:36:17.338419,m,1980,12855
33732,1_406,24.81,1,s_16790,c_1609,11.0,2021-04-06,11:53:14.590036,m,1980,12855


In [233]:
# Ajout de l'âge des clients
overview['age']=2022-overview.birth
overview

,id_prod,price,categ,session_id,client_id,hour,date,time,sex,birth,nb_sales_clients,age
0,0_2007,9.99,0,s_201,c_6906,10.0,2021-03-01,10:06:06.726895,m,1986,125,36
1,0_2096,14.84,0,s_311,c_6823,14.0,2021-03-01,14:54:32.544060,m,1958,37,64
2,0_1627,3.99,0,s_72,c_6823,2.0,2021-03-01,02:58:10.580555,m,1958,37,64
3,2_108,63.36,2,s_314,c_5042,15.0,2021-03-01,15:03:55.782366,m,1995,28,27
4,1_446,23.92,1,s_109,c_5890,4.0,2021-03-01,04:34:54.577506,m,1957,41,65
...,...,...,...,...,...,...,...,...,...,...,...,...
336811,0_1152,10.70,0,s_172145,c_332,10.0,2022-02-28,10:31:46.010138,f,1973,147,49
336812,0_2134,5.57,0,s_171999,c_903,3.0,2022-02-28,03:26:03.374741,f,1963,23,59
336813,0_2293,4.99,0,s_171940,c_2048,1.0,2022-02-28,01:05:47.900681,m,1987,84,35
336814,0_1140,3.73,0,s_171986,c_8547,2.0,2022-02-28,02:43:12.724027,f,1977,63,45


In [234]:
#Ajout de la colonne correspondant au mois
overview['month_per_year'] = overview['date'].apply(lambda x: x.strftime("%B"))
overview.head()

,id_prod,price,categ,session_id,client_id,hour,date,time,sex,birth,nb_sales_clients,age,month_per_year
0,0_2007,9.99,0,s_201,c_6906,10.0,2021-03-01,10:06:06.726895,m,1986,125,36,March
1,0_2096,14.84,0,s_311,c_6823,14.0,2021-03-01,14:54:32.544060,m,1958,37,64,March
2,0_1627,3.99,0,s_72,c_6823,2.0,2021-03-01,02:58:10.580555,m,1958,37,64,March
3,2_108,63.36,2,s_314,c_5042,15.0,2021-03-01,15:03:55.782366,m,1995,28,27,March
4,1_446,23.92,1,s_109,c_5890,4.0,2021-03-01,04:34:54.577506,m,1957,41,65,March


In [235]:
#Ajout du nombre de produits achetés par session
overview['basket_size']=overview.groupby('session_id')['id_prod'].transform('count')

In [236]:
#Ajout du montant des ventes par session
overview['basket_price']=overview.groupby('session_id')['price'].transform('sum')

In [237]:
#Ajout du nombre de sessions de ventes par mois
overview['sessions_month']=overview.groupby('month_per_year')['session_id'].transform('count')

In [238]:
#Ajout des tranches d'âge qui seront notamment nécessaires lors des calculs de corrélation
def f(x):
    if (x > 17) and (x <= 30):
        return '18-30'
    elif (x > 30) and (x <= 50 ):
        return '31-50'
    elif (x > 50) and (x <= 100):
        return'51+'
overview['age_group'] = overview['age'].apply(f)
overview

,id_prod,price,categ,session_id,client_id,hour,date,time,sex,birth,nb_sales_clients,age,month_per_year,basket_size,basket_price,sessions_month,age_group
0,0_2007,9.99,0,s_201,c_6906,10.0,2021-03-01,10:06:06.726895,m,1986,125,36,March,3,49.28,28568,31-50
1,0_2096,14.84,0,s_311,c_6823,14.0,2021-03-01,14:54:32.544060,m,1958,37,64,March,2,29.83,28568,51+
2,0_1627,3.99,0,s_72,c_6823,2.0,2021-03-01,02:58:10.580555,m,1958,37,64,March,1,3.99,28568,51+
3,2_108,63.36,2,s_314,c_5042,15.0,2021-03-01,15:03:55.782366,m,1995,28,27,March,1,63.36,28568,18-30
4,1_446,23.92,1,s_109,c_5890,4.0,2021-03-01,04:34:54.577506,m,1957,41,65,March,1,23.92,28568,51+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336811,0_1152,10.70,0,s_172145,c_332,10.0,2022-02-28,10:31:46.010138,f,1973,147,49,February,5,40.04,29567,31-50
336812,0_2134,5.57,0,s_171999,c_903,3.0,2022-02-28,03:26:03.374741,f,1963,23,59,February,1,5.57,29567,51+
336813,0_2293,4.99,0,s_171940,c_2048,1.0,2022-02-28,01:05:47.900681,m,1987,84,35,February,7,74.01,29567,31-50
336814,0_1140,3.73,0,s_171986,c_8547,2.0,2022-02-28,02:43:12.724027,f,1977,63,45,February,1,3.73,29567,31-50


In [239]:
#Conversion de la colonne "date" en datetime
overview['date']=pd.to_datetime(overview['date'])
overview.dtypes

id_prod                     object
price                      float64
categ                        int64
session_id                  object
client_id                   object
hour                       float64
date                datetime64[ns]
time                        object
sex                         object
birth                        int64
nb_sales_clients             int64
age                          int64
month_per_year              object
basket_size                  int64
basket_price               float64
sessions_month               int64
age_group                   object
dtype: object

In [242]:
#Exportation des fichiers finaux
overview.to_csv('overview_final.csv')
products.to_csv('products_final.csv')
transactions.to_csv('transactions_final.csv')
customers.to_csv('customers_final.csv')